In [31]:
import numpy as np
import pandas as pd
import scipy.optimize
from scipy import stats

import powerlaw
import pickle5 as pickle
import seaborn as sns
import matplotlib.pyplot as plt

# import fractal_working
import public_transport_synthetic_network as ptsn

In [2]:
#Unpickling the data
def load_obj(name):
    with open('resources/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
## Unchanged
def fractal_dimension(coords_data):
    """
    Graph may require some intuition to fit the linear regression through certain points
    """
    rangex = coords_data['x-coord'].values.max() - coords_data['x-coord'].values.min()
    rangey = coords_data['y-coord'].values.max() - coords_data['y-coord'].values.min()
    L = int(max(rangex, rangey)) # max of x or y distance

    r = np.array([ L/(2.0**i) for i in range(5,0,-1) ]) #Create array of box lengths
    N = [ fractal_working.count_boxes( coords_data, ri, L ) for ri in r ] #Non empty boxes for each array of box lenghts

    popt, pcov = scipy.optimize.curve_fit( fractal_working.f_temp, np.log( 1./r ), np.log( N ) )
    A, Df = popt #A lacunarity, Df fractal dimension

    return Df

In [9]:
# input data 
lsoa_data = load_obj("newdata_lsoa_data")
sheff_shape, income_params, edu_counts, edu_ratios = lsoa_data['sheff_lsoa_shape'], lsoa_data['income_params'], lsoa_data['edu_counts'], lsoa_data['edu_ratios']
comp_ratio = np.load("resources/newdata_companyhouse.npy")

# sampling attractivities 
attractivity_avg = ptsn.median_attractivity(edu_ratios, income_params)

# attractivity product - XiXj matrix 
attractivity_product = np.matmul(attractivity_avg, attractivity_avg.transpose())

# attractivity_product = np.multiply(attractivity_product, comp_ratio)

# population amplifier 
pop = np.asarray(edu_counts).reshape((len(edu_counts), 1))
pop = np.matmul(pop, pop.transpose())

# attractivity_product = np.multiply(attractivity_product, pop)
    
# distances
paths_matrix = load_obj("newdata_ave_paths")
# removes all 0s not on the diag 
paths_matrix[paths_matrix==0] = 1
paths_matrix[np.diag_indices_from(paths_matrix)] = 0
    
# current bus network
stoproute = pd.read_csv('resources/stoproute_withareacodes.csv')
lsoa_list = pd.read_csv("resources/E47000002_KS101EW.csv")['lsoa11cd']
route_freqs = pd.read_csv('resources/Bus_routes_frequency.csv', usecols= ["line","average"]).astype(str)
m_paths = ptsn.bus_adjacency(stoproute, lsoa_list, route_freqs)  # check if in main code the function is for the current network 
 

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)


In [33]:
centroids = euclidean_dists_fun(sheff_shape)
Df = ptsn.fractal_dimension(centroids)

base_m = 1
alpha = 1.45653
dc = base_m * (alpha - 1)

if Df <= dc:
    eta = ((-5/6) * Df) + dc
else:
    eta = (Df/6)

activity = np.power(paths_matrix, eta)

In [11]:
# attractivity product x population and company ratio amplificators 
amp_prod = np.multiply(attractivity_product, comp_ratio)
amp_prod =  np.multiply(attractivity_product, pop)

pat_prod = np.multiply(amp_prod, activity)

# amp_prod, pat_prod

In [22]:
# attractivity_product = np.multiply(attractivity_product, pop)
prod = pd.DataFrame(attractivity_product)
prod.index = sheff_shape.lsoa11cd.values
prod.columns = sheff_shape.lsoa11cd.values
product = pd.DataFrame(prod.stack())

# product = prod.copy()
product.columns = ['Attractivity product XiXj']

product

Attractivity product XiXj
E01007317 E01007317                  16.344367
          E01007318                  21.725865
          E01007319                  37.937627
          E01007320                  20.135743
          E01007321                  34.131049
...                                        ...
E01033280 E01033276                3224.255550
          E01033277                  65.330256
          E01033278                  75.606710
          E01033279                  63.158933
          E01033280                 182.948576

[727609 rows x 1 columns]

In [13]:
df_amp_prod = pd.DataFrame(amp_prod)
df_amp_prod.index = sheff_shape.lsoa11cd.values
df_amp_prod.columns = sheff_shape.lsoa11cd.values
df_amp_prod = pd.DataFrame(df_amp_prod.stack())

df_pat_prod = pd.DataFrame(pat_prod)
df_pat_prod.index = sheff_shape.lsoa11cd.values
df_pat_prod.columns = sheff_shape.lsoa11cd.values
df_pat_prod = pd.DataFrame(df_pat_prod.stack())

In [23]:
product['XiXj * pop and k'] = df_amp_prod
product['Amplified product * l^eta'] = df_pat_prod

product

Attractivity product XiXj  XiXj * pop and k  \
E01007317 E01007317                  16.344367      3.643239e+07   
          E01007318                  21.725865      5.008229e+07   
          E01007319                  37.937627      8.297888e+07   
          E01007320                  20.135743      4.798001e+07   
          E01007321                  34.131049      7.302232e+07   
...                                        ...               ...   
E01033280 E01033276                3224.255550      1.542790e+10   
          E01033277                  65.330256      2.496111e+08   
          E01033278                  75.606710      3.008805e+08   
          E01033279                  63.158933      2.458642e+08   
          E01033280                 182.948576      4.902599e+08   

                     Amplified product * l^eta  
E01007317 E01007317               0.000000e+00  
          E01007318               2.876783e+08  
          E01007319               7.033863e+08  
          E01007320               3.073433e+08  
          E01007321               6.250453e+08  
...                                        ...  
E01033280 E01033276               1.569247e+11  
          E01033277               2.481531e+09  
          E01033278               3.863773e+09  
          E01033279               3.184134e+09  
          E01033280               0.000000e+00  

[727609 rows x 3 columns]

In [24]:
product_sorted = product.sort_values(['Attractivity product XiXj'], ascending = False)

product_sorted

Attractivity product XiXj  XiXj * pop and k  \
E01033261 E01033261               58581.472493      1.829083e+11   
          E01033276               57695.917133      2.979960e+11   
E01033276 E01033261               57695.917133      2.979960e+11   
          E01033276               56823.748400      4.854981e+11   
E01033261 E01008067               53676.405202      1.597210e+11   
...                                        ...               ...   
E01007970 E01007629                   1.000000      2.532662e+06   
E01007947 E01007333                   1.000000      2.164206e+06   
E01007543 E01007543                   1.000000      2.250000e+06   
E01007378 E01007543                   1.000000      2.358000e+06   
          E01007739                   1.000000      2.494764e+06   

                     Amplified product * l^eta  
E01033261 E01033261               0.000000e+00  
          E01033276               1.944875e+12  
E01033276 E01033261               1.846041e+12  
          E01033276               0.000000e+00  
E01033261 E01008067               1.105570e+12  
...                                        ...  
E01007970 E01007629               4.345959e+07  
E01007947 E01007333               3.193464e+07  
E01007543 E01007543               0.000000e+00  
E01007378 E01007543               2.725105e+07  
          E01007739               3.276767e+07  

[727609 rows x 3 columns]

In [26]:
product_sorted.to_csv('Attractivity product weighting', encoding='utf-8', index=True)

# df.to_csv(file_name, encoding='utf-8', index=False)